In [103]:
import numpy as np

nfl = 3
ndir = 3
corr = []

#Reading inputs
with open("meson_corrs", "r") as f:
    read_next = False
    Nt = 0
    tot = 0
    for line in f:
        if ("iop 2" in line or "iop 3" in line or "iop 4" in line):
            read_next = True 
        elif ("#" in line and ("iop 2" not in line or "iop 3" not in line or "iop 4" not in line)):
            read_next = False
        elif read_next:
            if not line.strip():
                Nt = 0;
            if line.strip():
                columns = line.split()
                corr.append(columns[1])
                Nt += 1
                tot +=1
                #print(columns[1])
                
             
NConf = int(tot/(nfl*ndir*count))
print("Nt =", Nt,", NConf =",NConf, ", TotDim = ", tot)
print(len(corr))

#Dividing them in categories 
corr_2 = np.zeros((int(NConf), Nt, nfl, ndir))
for c in range(int(NConf)):
    for i in range(Nt):
        for j in range(nfl):
            for k in range(ndir):
                corr_2[c][i][j][k] = corr[c*Nt*nfl*ndir + i*nfl*ndir + j*ndir + k]

            
print("Dimensions of corr_2: ", len(corr_2), " x ", len(corr_2[0]), " x ", len(corr_2[0][0]), " x ", len(corr_2[0][0][0]))

print(corr_2[0][0][0][0], corr_2[55][9][2][2])

Nt = 10 , NConf = 56 , TotDim =  5040
5040
Dimensions of corr_2:  56  x  10  x  3  x  3
-0.3889598401751052 -0.1412132072258582


In [124]:
#Bootstrap resampling
import tensorflow as tf


Nboot = 1000
corr_boot = np.zeros((Nboot, int(NConf), Nt, nfl, ndir))

b_index = tf.random.uniform([Nboot, NConf, Nt, nfl, ndir], minval=0, maxval=NConf-1, dtype=tf.int32).numpy()
print(b_index.shape)
print(np.amin(b_index), np.amax(b_index))

for b in range(Nboot):
    for c in range(int(NConf)):
        for i in range(Nt):
            for j in range(nfl):
                for k in range(ndir):
                    corr_boot[b][c][i][j][k] = corr_2[b_index[b][c][i][j][k]][i][j][k]

(1000, 56, 10, 3, 3)
0 54


In [177]:
from scipy import stats
#Operations on the array

#Average on directions
b = np.mean(corr_boot, axis=4)
print(b.shape)

#Weighted sum on flavours
weights= np.array((4/9, 1/9, 1/9))
print(weights)
bm = np.tensordot(b, weights, axes=(-1, 0))
print(bm.shape)

#Average on configurations
cbfin = np.mean(bm, axis=1)
print(cbfin.shape)

#Bootstrap Mean and Sigma
def mean(arr):
    return sum(arr)/len(arr)
def std_dev(arr):
    arr_mean = mean(arr)
    return (sum([(i - arr_mean)**2 for i in arr]) / (len(arr)-1))**0.5

cmean = np.zeros(10)
csigma = np.zeros(10)
for i in range(Nt):
    cmean[i] = mean(cbfin[:,i])
    csigma[i] = std_dev(cbfin[:,i])

print(cmean)
print(csigma)

(1000, 56, 10, 3)
[0.44444444 0.11111111 0.11111111]
(1000, 56, 10)
(1000, 10)
[-0.08368885 -0.10212141 -0.08872146 -0.04666328 -0.03131613 -0.03032359
 -0.02839207 -0.02972161 -0.02639963 -0.0321877 ]
[3.69402502e-05 3.77517418e-05 3.30232414e-05 1.58507053e-05
 2.10762627e-05 1.83451104e-05 1.01629565e-05 1.77565091e-05
 1.80778765e-05 1.33315663e-05]
